<a href="https://colab.research.google.com/github/me-t1me/DIP_assignment_2/blob/master/DIP_2_Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import mean_squared_error as mse

In [2]:
!wget "https://raw.githubusercontent.com/me-t1me/DIP_assignment_2/master/xray.png"
!wget "https://raw.githubusercontent.com/me-t1me/DIP_assignment_2/master/xray_sp.png"

--2023-04-29 04:33:07--  https://raw.githubusercontent.com/me-t1me/DIP_assignment_2/master/xray.png
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 369053 (360K) [image/png]
Saving to: ‘xray.png’

xray.png            100%[===================>] 360.40K  --.-KB/s    in 0.01s   

2023-04-29 04:33:07 (28.5 MB/s) - ‘xray.png’ saved [369053/369053]

--2023-04-29 04:33:07--  https://raw.githubusercontent.com/me-t1me/DIP_assignment_2/master/xray_sp.png
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 397248 (388K) [image/png]
Saving to: ‘x

In [3]:
# Load the original X-ray image
img = cv2.imread('xray.png', 0)

# Load the salt & pepper noise-based contaminated X-ray image
img_sp = cv2.imread('xray_sp.png', 0)

In [9]:
# Apply median filter to the contaminated X-ray image
img_median = cv2.medianBlur(img_sp, 3)

# Apply max filter to the contaminated X-ray image
kernel = np.ones((3, 3), np.uint8)
img_max = cv2.dilate(img_sp, kernel, iterations=1)

# Apply min filter to the contaminated X-ray image
img_min = cv2.erode(img_sp, kernel, iterations=1)

# Apply alpha-trimmed mean filter to the contaminated X-ray image
def alpha_trimmed_mean_filter(image, kernel_size=3, alpha=0.5):
    # Pad the image with zeros
    padded = cv2.copyMakeBorder(image, kernel_size // 2, kernel_size // 2,
                                kernel_size // 2, kernel_size // 2,
                                cv2.BORDER_CONSTANT, value=0)

    # Loop over all pixels in the image
    filtered = np.zeros_like(image)
    for y in range(kernel_size // 2, image.shape[0] + kernel_size // 2):
        for x in range(kernel_size // 2, image.shape[1] + kernel_size // 2):
            # Extract the kernel centered at the current pixel
            kernel = padded[y - kernel_size // 2:y + kernel_size // 2 + 1,
                            x - kernel_size // 2:x + kernel_size // 2 + 1]

            # Sort the pixel values and remove the specified number of trimmed pixels
            sorted_kernel = np.sort(kernel.flatten())
            trimmed_kernel = sorted_kernel[int(alpha*kernel_size*kernel_size) : -1*int(alpha*kernel_size*kernel_size)]

            # Compute the mean of the remaining pixel values
            filtered[y - kernel_size // 2, x - kernel_size // 2] = np.mean(trimmed_kernel)

    return filtered.astype(np.uint8)

img_at_mean = alpha_trimmed_mean_filter(img_sp)
# img_at_mean = cv2.boxFilter(img_sp, -1, (5,5), normalize=False)

# Evaluate the SNR and RMSE values for the filtered images using the original X-ray image as a reference
snr_median = psnr(img, img_median)
rmse_median = np.sqrt(mse(img, img_median))

snr_max = psnr(img, img_max)
rmse_max = np.sqrt(mse(img, img_max))

snr_min = psnr(img, img_min)
rmse_min = np.sqrt(mse(img, img_min))

snr_at_mean = psnr(img, img_at_mean)
rmse_at_mean = np.sqrt(mse(img, img_at_mean))

print("SNR Median: ", snr_median)
print("RMSE Median: ", rmse_median)

print("SNR Max: ", snr_max)
print("RMSE Max: ", rmse_max)

print("SNR Min: ", snr_min)
print("RMSE Min: ", rmse_min)

print("SNR Alpha-Trimmed Mean: ", snr_at_mean)
print("RMSE Alpha-Trimmed Mean: ", rmse_at_mean)


SNR Median:  45.86795333323992
RMSE Median:  1.2976050103237822
SNR Max:  19.846847585896857
RMSE Max:  25.95361160474386
SNR Min:  22.13191991959036
RMSE Min:  19.95005900247592
SNR Alpha-Trimmed Mean:  47.723691899767466
RMSE Alpha-Trimmed Mean:  1.0479862521194387
